<div><img src="https://radar.community.uaf.edu/wp-content/uploads/sites/667/2021/03/HydroSARbanner.jpg" width="100%" /></div>

**NASA A.37 Project:** Integrating SAR Data for Improved Resilience and Response to Weather-Related Disasters
**PI:** Franz J. Meyer

# HydroSAR Transition workshop

## Monitoring the HKH region

In order to automatically create new HydroSAR products for an Area of Interest (AOI), we need to regularly *monitor* an AOI for *newly* acquired Sentinel-1 scenes. To do so, we:
1. Search our AOI for *all* possible scenes to process
2. Look up the scenes we've already processed
3. Deduplicate the scene lists to determine the *new* scenes to process
4. Submit the new scenes for processing

This notebook walks through the monitoring process and would be run on a regular schedule (cron) in application.

Note: Here we're taking the strategy to *always* search for **all** possible scenes, *always* look up **all** previously processed scenes, and then *deduplicate* the two lists to determine the new scenes. You could instead keep track of the last time the script was *successfully* run and only search for new scenes since then. While this would be more performant, our strategy is independent of previous runs so is generally more fault-tolerant and can be started and stopped at will.

### 1. Search for all possible scenes to process

First, we define our Area Of Interest (AOI) as a WellKnow Text (WKT) polygon

In [ ]:
area_of_interest = "POLYGON((88.2733 26.8522,87.9897 26.8555,87.9767 26.8534,87.3067 26.7491,87.0011 26.8575,86.9938 26.8575,84.6891 27.6531,82.5953 28.1529,81.5007 28.7001,80.0969 29.0809,79.1861 29.615,78.5167 28.5766,78.7787 27.2874,78.6919 26.4344,78.6205 25.3707,78.5149 24.2368,78.5763 23.8111,78.8046 23.5573,79.2323 22.9953,79.3721 22.9435,79.5727 22.9467,80.1637 23.1417,80.5249 23.2157,80.9465 23.2017,81.7853 22.8307,81.8401 22.6401,82.0433 22.5511,82.8631 22.5995,82.8712 22.6001,82.8713 22.5989,83.0577 22.5431,83.2023 22.6805,83.1085 22.8209,83.3609 23.1285,83.9519 23.3525,84.1695 23.6465,84.4725 23.5153,84.6093 23.5967,84.8117 23.9187,84.9515 23.9705,85.0393 23.8681,85.2067 23.9395,85.6463 24.5699,85.8413 24.5311,85.9189 24.3827,86.1818 24.2755,86.1934 24.268,87.0539 23.8361,87.7399 23.5313,88.0203 23.2965,88.0786 23.0512,87.9981 22.2447,87.9641 21.9236,87.9821 21.7886,87.9975 21.6392,88.191 21.5641,88.2556 21.5643,88.2646 21.5644,88.8358 21.5667,90.1433 21.7556,90.6789 21.7432,91.1617 22.0114,91.5084 22.2822,91.7269 22.7256,91.9075 23.1854,92.0615 23.4098,92.1437 23.4281,92.4821 23.2537,92.7431 22.7425,92.7835 22.6984,92.8256 22.7276,92.8537 22.7658,93.5325 23.6863,93.7665 24.1093,93.8401 24.2885,94.1175 24.9533,94.1589 25.1475,94.4519 25.3565,94.5485 25.7317,94.7423 25.7553,94.8831 25.8907,95.0069 26.2627,94.9327 26.5995,95.0499 26.7365,95.5617 26.9655,96.6105 26.9397,96.7527 27.0529,96.7275 27.2537,96.7207 27.3849,96.7199 27.5913,96.6269 27.7277,96.4177 27.7913,95.5395 28.0577,95.0637 27.7723,93.5647 27.1297,92.0893 26.8205,91.9249 26.8092,90.7093 26.7253,89.7337 26.7253,88.8211 26.8469,88.4961 26.8513,88.2733 26.8522))"

And then we define our additional search parameters. HyP3 processing currently accepts Sentinel-1 SLC or GRDH scenes acquired with the IW beam mode, for RTC and HYDRO processing. The water mapping algorithm additionally requires the VV and VH polarizations. And finally, we want to monitor our AOI over the course of the entire monsoon season, with a buffer of time on both sides of the monsoon season.

In [ ]:
search_parameters = {
    "platform": ["S1"],
    "processingLevel": ["SLC"],
    "beamMode": ["IW"],
    "polarization": ["VV+VH"],
    "start": "2022-05-18",
    "end": "2022-12-01",
    "intersectsWith": area_of_interest
}

Now that we have our search parameters defined, we can use the [ASF Search Python Package](https://docs.asf.alaska.edu/asf_search/basics/) to search ASF DAAC's Archive for scenes to process and build a set of the scene names.

In [ ]:
import asf_search as asf

search_results = asf.geo_search(**search_parameters)
all_scenes = {result.geojson()['properties']['sceneName'] for result in search_results}
print(f'Found {len(all_scenes)} scenes')

### 2. Look up the scenes we've already processed

We use HyP3 as our workflow engine and can query for all the scenes we've already processed. Importantly, when we submit jobs, we will assign all of them a project name which is used to group jobs together and later search for them.

In [ ]:
project_name = 'HKHwatermaps'

In this notebook, we'll prompt for an Earthdata Login username and password, but they can be provided via the `username` and `password` keyword arguments, or automatically pulled from the users `~/.netrc` file.

Note: Typically you'll want to use a shared "operational" Earthdata Login user as you can only search for jobs associated with your username.

In [ ]:
import hyp3_sdk as sdk
hyp3 = sdk.HyP3('https://hyp3-watermap.asf.alaska.edu', prompt=True)

Now we'll search for all the jobs with our project name and build a set of the processed scene names.

In [ ]:
processed_jobs = hyp3.find_jobs(name=project_name)
processed_scenes = {job.job_parameters['granules'][0] for job in processed_jobs}
print(f'We have already processed {len(processed_scenes)} scenes')

### 3. Deduplicate the scene lists to determine the *new* scenes to process

In the simplest sense, this is just a set operation where we remove the processed scenes from all the scenes, leaving only the new scenes to process.

Importantly, we're explicitly not filtering out running or failed jobs from the project name search, so we also won't re-submit running jobs or failed jobs.

In [ ]:
scenes_to_process = all_scenes - processed_scenes
print(f'There are {len(scenes_to_process)} new scenes to process')

### 4. Submit the new scenes for processing

Now that we have our scenes to submit for processing, we need to define our processing options. `WATER_MAP` HyP3 jobs will:
1. Radiometerically Terrain Correct (RTC) the input scene
2. Create a surface water extent map (WM) from the RTC products
3. Create water and flood depth maps (FD) from the WM and RTC products

Below are the default parameters used for the HydroSAR products, with a comment for which part of the processing the option applies to.

In [ ]:
job_spec = {
    "job_type": "WATER_MAP",
    "job_name": project_name,
    "job_parameters": {
          "resolution": 30,              # RTC
          "speckle_filter": True,        # RTC
          "max_vv_threshold": -15.5,     # WM
          "max_vh_threshold": -23.0,     # WM
          "hand_threshold": 15.0,        # WM
          "hand_fraction": 0.8,          # WM
          "membership_threshold": 0.45,  # WM
          "include_flood_depth": True,   # FD
          "iterative_max": 15,           # FD
          "iterative_min": 0,            # FD
          "known_water_threshold": 30,   # FD
          "water_level_sigma": 3         # FD
    }
}

Now, we can use the above parameters to prapare a list of jobs to submit

In [ ]:
import copy

prepared_jobs = []
for scene in scenes_to_process:
    job_dict = copy.deepcopy(job_spec)
    job_dict['job_parameters']['granules'] = [scene]
    prepared_jobs.append(job_dict)

and submit them to HyP3.

Note: the HyP3 API only accepts submitting 200 jobs at a time, so we use the `chunk` utility to break up the submission into sets of 200.

In [ ]:
from hyp3_sdk.util import chunk

submitted_jobs = sdk.Batch()
for batch in chunk(prepared_jobs):
    submitted_jobs += hyp3.submit_prepared_jobs(batch)

Now they are off and running! We can use the `HyP3.watch` method to track the jobs progress and wait for them to finish.

In [ ]:
submitted_jobs = hyp3.watch(submitted_jobs)

## Operations

To do all this operationally, you'll typically want to extract all the above configuration into a separate configuration file, and the python code into an easily executable script, and then execute the script regularly via a cron job (or similar) *somewhere*.

In practice, ASF:
* Has defined all the configuration in this file: https://github.com/ASFHyP3/hyp3-nasa-disasters/blob/main/data_management/hkh_watermaps.json
* Scripted this notebook: https://github.com/ASFHyP3/hyp3-nasa-disasters/blob/main/data_management/process_new_granules.py
* Executes the script every 6 hours using GitHub Actions "scheduled" (cron) workflows: https://github.com/ASFHyP3/hyp3-nasa-disasters/blob/main/.github/workflows/process-and-transfer.yml
